In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import numpy as np
import tensorflow as tf

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [3]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

In [4]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [5]:
#Read in csv and create a dataset
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
    return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.


In [6]:
#Each item in dataset is a batch, represented as a tuple.
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))
            
#Pretty print the dataset.
show_batch(raw_train_data)            

sex                 : [b'female' b'female' b'male' b'male' b'female']
age                 : [30. 22. 28. 30. 27.]
n_siblings_spouses  : [0 1 0 0 0]
parch               : [0 2 0 0 0]
fare                : [56.929 41.579  7.733 13.    10.5  ]
class               : [b'First' b'Second' b'Third' b'Second' b'Second']
deck                : [b'E' b'unknown' b'unknown' b'unknown' b'E']
embark_town         : [b'Cherbourg' b'Cherbourg' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'y']


In [7]:
#Manually pass in column names if dataset constructor doesn't get automatically.
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 : [b'female' b'male' b'male' b'male' b'female']
age                 : [31. 39. 25. 28. 14.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [0 0 0 0 0]
fare                : [ 8.683 24.15   7.05   0.     7.854]
class               : [b'Third' b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'y']


In [8]:
#Select subset of available columns
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [28.    0.75 45.   28.   34.  ]
n_siblings_spouses  : [0 2 1 0 1]
class               : [b'Third' b'Third' b'First' b'Third' b'Second']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'n' b'n' b'y' b'n']


In [9]:
#Continuous data
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, 
                           select_columns=SELECT_COLUMNS,
                           column_defaults = DEFAULTS)

show_batch(temp_dataset)

age                 : [28. 29.  9. 19. 28.]
n_siblings_spouses  : [0. 0. 2. 0. 0.]
parch               : [0. 4. 2. 0. 0.]
fare                : [ 7.225 21.075 34.375  8.05   7.75 ]


In [10]:
example_batch, labels_batch = next(iter(temp_dataset)) 

In [11]:
#pack together all the columns
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

#apply to each element of dataset
packed_dataset = temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy(),'\n')
    print(labels.numpy())

[[28.     0.     0.     7.896]
 [27.     0.     2.    11.133]
 [ 2.     4.     1.    39.688]
 [ 2.     1.     1.    26.   ]
 [28.     1.     0.    24.   ]] 

[0 1 0 1 1]


In [12]:
#Switch back to the mixed dataset
show_batch(raw_train_data)

sex                 : [b'female' b'female' b'male' b'male' b'male']
age                 : [30. 19. 28. 34. 23.]
n_siblings_spouses  : [0 0 0 1 0]
parch               : [0 0 0 0 0]
fare                : [ 8.663 26.     7.75  21.     7.854]
class               : [b'Third' b'Second' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Queenstown' b'Southampton' b'Southampton']
alone               : [b'y' b'y' b'y' b'n' b'y']


In [13]:
example_batch, labels_batch = next(iter(temp_dataset))

In [14]:
#class that packs list of numeric features in one column
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names

    def __call__(self, features, labels):
        numeric_freatures = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_freatures]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features

        return features, labels

In [15]:
#pack list of numeric features in one column
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']

packed_train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

packed_test_data = raw_test_data.map(
    PackNumericFeatures(NUMERIC_FEATURES))

In [16]:
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
class               : [b'First' b'Third' b'First' b'First' b'Third']
deck                : [b'unknown' b'unknown' b'C' b'C' b'unknown']
embark_town         : [b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton' b'Queenstown']
alone               : [b'y' b'n' b'y' b'n' b'y']
numeric             : [[ 28.      0.      0.     39.6  ]
 [ 41.      2.      0.     14.108]
 [ 30.      0.      0.     27.75 ]
 [ 64.      1.      4.    263.   ]
 [ 28.      0.      0.      7.75 ]]


In [17]:
example_batch, labels_batch = next(iter(packed_train_data)) 

In [18]:
import pandas as pd
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [19]:
MEAN = np.array(desc.T['mean'])  #list of mean using desc above
STD = np.array(desc.T['std'])    ##list of std using desc above

In [20]:
def normalize_numeric_data(data, mean, std):
  # Center the data
  return (data-mean)/std

In [21]:
# See what you just created.
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)

numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]   #Python list
numeric_column                       #tensorflow NumericColumn

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x000001E7560A92F0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [22]:
example_batch['numeric']

<tf.Tensor: id=550, shape=(5, 4), dtype=float32, numpy=
array([[ 44.   ,   0.   ,   0.   ,   7.925],
       [ 49.   ,   1.   ,   0.   ,  76.729],
       [ 28.   ,   0.   ,   0.   ,   7.75 ],
       [ 25.   ,   1.   ,   0.   ,  17.8  ],
       [ 28.   ,   1.   ,   0.   , 146.521]], dtype=float32)>

In [23]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[ 1.148, -0.474, -0.479, -0.485],
       [ 1.548,  0.395, -0.479,  0.776],
       [-0.13 , -0.474, -0.479, -0.488],
       [-0.37 ,  0.395, -0.479, -0.304],
       [-0.13 ,  0.395, -0.479,  2.054]], dtype=float32)

### Categorical data

In [24]:
#create categorical columns
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))
    
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southhampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [25]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


### Combined preprocessing layer

In [26]:
# Add the two feature column lists and pass them to a tf.keras.layers.DenseFeatures 
# to create an input layer that will extract and preprocess both input types.

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)
print(preprocessing_layer(example_batch).numpy()[0])

[ 1.     0.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     1.148 -0.474
 -0.479 -0.485  1.     0.   ]


### Build the Model

In [27]:
#Build a tf.keras.Sequential, starting with the preprocessing_layer.

model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

### Train, evaluate, and predict

In [28]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data

In [29]:
#train
print('model training:')
model.fit(train_data, epochs=20)

model training:
Epoch 1/20
126/126 [==============================] - 2s 15ms/step - loss: 0.4969 - accuracy: 0.7624
Epoch 2/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4216 - accuracy: 0.8230
Epoch 3/20
126/126 [==============================] - 1s 6ms/step - loss: 0.4038 - accuracy: 0.8230
Epoch 4/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3966 - accuracy: 0.8357
Epoch 5/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3812 - accuracy: 0.8262
Epoch 6/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3728 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3589 - accuracy: 0.8501
Epoch 8/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3672 - accuracy: 0.8421
Epoch 9/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3497 - accuracy: 0.8517
Epoch 10/20
126/126 [==============================] - 1s 6ms/step - loss: 0.3419 -

In [30]:
# Once the model is trained, you can check its accuracy on the test_data set.
print('test loss & accuracy:')
test_loss, test_accuracy = model.evaluate(test_data)
print(f'\n\nTest Loss {test_loss}, Test Accuracy {test_accuracy}')

test loss & accuracy:
53/53 [==============================] - 1s 9ms/step - loss: 0.4761 - accuracy: 0.8409.5102 - accura


Test Loss 0.4761135882504706, Test Accuracy 0.8409090638160706


In [31]:
print(f'data type of test_data:  {type(test_data)}')

data type of test_data:  <class 'tensorflow.python.data.ops.dataset_ops.MapDataset'>


In [32]:
#Use tf.keras.Model.predict to infer labels on a batch or a dataset of batches.

predictions = model.predict(test_data)
print('predictions shape:', predictions.shape,'\n')

# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
    print("Predicted survival: {:.1%}".format(prediction[0]), " | True outcome: ",("SURVIVED" if bool(survived) else "DIED"))

predictions shape: (264, 1) 

Predicted survival: 99.5%  | True outcome:  DIED
Predicted survival: 7.3%  | True outcome:  DIED
Predicted survival: 97.9%  | True outcome:  DIED
Predicted survival: 12.7%  | True outcome:  DIED
Predicted survival: 42.3%  | True outcome:  SURVIVED
